In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from neuralforecast.models import TimesNet
from neuralforecast.losses.pytorch import DistributionLoss
from neuralforecast import NeuralForecast

# Load the data
df = pd.read_csv('/content/drive/MyDrive/Dataset BA/DXG.csv')

# Preprocess the data
# Hàm chuyển đổi "Volume" từ chuỗi sang số
def convert_volume_to_number(volume_str):
    if volume_str[-1] == 'K':
        return int(float(volume_str[:-1]) * 1000)
    elif volume_str[-1] == 'M':
        return int(float(volume_str[:-1]) * 1000000)
    elif volume_str[-1] == 'B':
        return int(float(volume_str[:-1]) * 1000000000)
    else:
        return int(volume_str)

df["Volume"] = df["Volume"].apply(convert_volume_to_number)


# Hàm chuyển đổi "Change %" từ chuỗi sang số
def convert_change_to_number(change_str):
    new_change = float(change_str.strip('%')) / 100
    return new_change

df["Change %"] = df["Change %"].apply(convert_change_to_number)

# Hàm chuyển giá từ chuỗi sang số
def convert_str_to_number(str):
    return float(str.replace(',', ''))

df["Close"] = df["Close"].apply(convert_str_to_number)
df["Open"] = df["Open"].apply(convert_str_to_number)
df["High"] = df["High"].apply(convert_str_to_number)
df["Low"] = df["Low"].apply(convert_str_to_number)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)  # Chuyển đổi cột date thành định dạng thời gian
df.set_index('Date', inplace=True)  # Đặt cột date làm index

# Check for null values and drop them
null_count = df.isnull().sum()
print(null_count)
df = df.dropna()

# Normalize the data
scaler = MinMaxScaler()
df['Close'] = scaler.fit_transform(df[['Close']])

# Tăng cường dữ liệu lịch (nếu cần)
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['day_of_year'] = df.index.dayofyear
df['day_of_month'] = df.index.day
df['week_of_year'] = df.index.isocalendar().week

# Tạo danh sách các cột lịch
calendar_cols = ['day_of_week', 'month', 'day_of_year', 'day_of_month', 'week_of_year']

# Chuyển đổi dữ liệu thành định dạng yêu cầu bởi NeuralForecast
df.reset_index(inplace=True)
df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
df['unique_id'] = 'DXG'

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_size = int(len(df) * 0.9)
val_size = int(len(df) * 0.05)
Y_train_df = df[:train_size]
Y_val_df = df[train_size:train_size+val_size]
Y_test_df = df[train_size+val_size:].reset_index(drop=True)

# Không có dữ liệu tĩnh, tạo DataFrame trống
DXGStatic = pd.DataFrame({'unique_id': ['DXG'], 'static_feature': [1]})

# Build the model
model = TimesNet(
    h=12,
    input_size=24,
    hidden_size=16,
    conv_hidden_size=32,
    loss=DistributionLoss(distribution='Normal', level=[80, 90]),
    futr_exog_list=calendar_cols,
    scaler_type='standard',
    learning_rate=1e-3,
    max_steps=5,
    val_check_steps=50,
    early_stop_patience_steps=2
)

# Initialize NeuralForecast
nf = NeuralForecast(
    models=[model],
    freq='M'
)

# Fit the model
nf.fit(df=Y_train_df, static_df=DXGStatic, val_size=12)

# Generate future dates for prediction
future_dates = pd.date_range(start=Y_test_df['ds'].max(), periods=30, freq='D')
futr_df = pd.DataFrame({
    'unique_id': 'DXG',
    'ds': future_dates
})

# Get missing combinations
missing_combinations = nf.get_missing_future(futr_df)

# Nếu có kết hợp thiếu, thêm chúng vào futr_df
if not missing_combinations.empty:
    futr_df = pd.concat([futr_df, missing_combinations], ignore_index=True)

# Add calendar columns to futr_df
futr_df['day_of_week'] = futr_df['ds'].dt.dayofweek
futr_df['day_of_year'] = futr_df['ds'].dt.dayofyear
futr_df['day_of_month'] = futr_df['ds'].dt.day
futr_df['week_of_year'] = futr_df['ds'].dt.isocalendar().week
futr_df['month'] = futr_df['ds'].dt.month / 12

# Predict future prices
forecasts = nf.predict(futr_df=futr_df)

# Plot the results
# Plot the results
plt.figure(figsize=(14, 7))

# Plot the results
plt.figure(figsize=(14, 7))

# True data
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')

# Train, Validation, Test splits
plt.axvline(x=Y_train_df['ds'].max(), color='green', linestyle='--', label='Train End')
plt.axvline(x=Y_val_df['ds'].max(), color='red', linestyle='--', label='Validation End')
plt.axvline(x=Y_test_df['ds'].min(), color='blue', linestyle='--', label='Test End')

# Predictions for the next 30 days
plt.plot(forecasts['ds'], forecasts['TimesNet-median'], c='blue', label='Median Prediction (Next 30 Days)')
plt.fill_between(x=forecasts['ds'],
                 y1=forecasts['TimesNet-lo-90'].values,
                 y2=forecasts['TimesNet-hi-90'].values,
                 alpha=0.4, color='orange', label='90% Confidence Interval (Next 30 Days)')

plt.title('DXG Stock Price Prediction with 30-day Forecast')
plt.xlabel('Date')
plt.ylabel('Normalized Close Price')
plt.legend()
plt.grid()
plt.show()




ModuleNotFoundError: No module named 'neuralforecast'

# New Section